In [2]:
import copy
import cv2
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import random
import face_recognition
from tqdm import tqdm 

In [150]:
bb_dimensions = (40, 40)

def get_bounding_box(data):
    min_x = min(data, key = lambda t: t[0])[0]
    max_x = max(data, key = lambda t: t[0])[0]
    min_y = min(data, key = lambda t: t[1])[1]
    max_y = max(data, key = lambda t: t[1])[1]

    offset_x = int((bb_dimensions[0] - (max_x - min_x)) / 2.0)
    offset_y = int((bb_dimensions[1] - (max_y - min_y)) / 2.0)

    bb_tl = (min_x - offset_x, min_y - offset_y)
    bb_bl = (min_x - offset_x, max_y + offset_y)
    bb_tr = (max_x + offset_x, min_y - offset_y)
    bb_br = (max_x + offset_x, max_y + offset_y)

    return [bb_tl, bb_tr, bb_br, bb_bl]

def get_pixels(I, bb):
    top_left = bb[0]
    pixels = np.zeros((bb_dimensions[0], bb_dimensions[1], 3), dtype=np.uint8)
    for i in range(bb_dimensions[0]):
        for j in range(bb_dimensions[1]):
            new_top_left_y = top_left[1]+ j
            new_top_left_x = top_left[0] + i
            if new_top_left_y > 127: 
               new_top_left_y = 127
            if new_top_left_x > 127: 
               new_top_left_x = 127
            pixels[j, i, :] = I[new_top_left_y, new_top_left_x, :]
    return pixels

# image: numpy ndarray (e.g. from cv2.imread)
def get_feature_vector(image):

    face_landmarks = face_recognition.face_landmarks(image)
    if len(face_landmarks) == 0:
      return []
    face_landmarks = face_recognition.face_landmarks(image)[0]
    eyel = get_pixels(image, get_bounding_box(face_landmarks['left_eye']))
    #print('eyel')
    eyer = get_pixels(image, get_bounding_box(face_landmarks['right_eye']))
    #print('eyer')
    nose = get_pixels(image, get_bounding_box(face_landmarks['nose_bridge'] + face_landmarks['nose_tip']))
    #print('nose')
    lips = get_pixels(image, get_bounding_box(face_landmarks['top_lip'] + face_landmarks['bottom_lip']))
    #print('lips')
    feature_vector = np.concatenate((eyel.ravel(), eyer.ravel(), nose.ravel(), lips.ravel()))
    return feature_vector

In [147]:
def get_feature_array_from_image_folder(imgfolder):
    imgpathlist_new = []
    images_resized = []
    features_all = []
    for img in tqdm(os.listdir(imgfolder)):
        img_path = os.path.join(imgfolder, img)
        if (not img.startswith('.')) and (os.path.isfile(img_path)):            
            #img_cv2 = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img_cv2 = cv2.imread(img_path)[...,::-1]
            img_cv2_res = cv2.resize(img_cv2, (128, 128), interpolation = cv2.INTER_AREA)
            feature_vector = get_feature_vector(img_cv2_res)
            if len(feature_vector) == 0:
                continue
            features_all.append(feature_vector)
            imgpathlist_new.append(img)
    return imgpathlist_new, np.asarray(features_all)

In [151]:
from tqdm import tqdm

def get_feature_array_from_image_list(imgpathlist):
    imgpathlist_new = []
    images_resized = []
    features_all = []
    for img in tqdm(imgpathlist):
        print(img)
        img_cv2 = cv2.imread(img)[...,::-1]
        img_cv2_res = cv2.resize(img_cv2, (128, 128), interpolation = cv2.INTER_AREA)
        feature_vector = get_feature_vector(img_cv2_res)
        if len(feature_vector) == 0:
          continue
        features_all.append(feature_vector)
        imgpathlist_new.append(img)
    return imgpathlist_new, np.asarray(features_all)

In [190]:
def get_feature_array_from_image_list_newpath(imgpathlist, newpath):
    imgpathlist_colab = []
    images_resized = []
    features_all = []
    for img in tqdm(imgpathlist):
        img_new = get_new_path_for_image(img, newpath)
        img_cv2 = cv2.imread(img_new)[...,::-1]
        img_cv2_res = cv2.resize(img_cv2, (128, 128), interpolation = cv2.INTER_AREA)
        feature_vector = get_feature_vector(img_cv2_res)
        if len(feature_vector) == 0:
          continue
        features_all.append(feature_vector)
        imgpathlist_colab.append(img)
    return imgpathlist_colab, np.asarray(features_all)

In [152]:
def get_new_path_for_image(imgpathname, path):
    imgname = imgpathname.split("/")[-1]
    imgpathname_new = path+imgname
    return imgpathname_new

In [156]:
def get_fake_real_tvt_facenames(dataset, fake_real, train_test_val):
    dataset = dataset[~dataset['face_path_new'].isnull()]
    dataset = dataset[dataset['fake_real_name'] == fake_real]
    dataset = dataset[dataset['train'] ==  train_test_val]
    facenames = dataset['face_path_new'].tolist()
    return facenames

In [186]:
!ls "./data/ruska/deepfakes/faces" -l |wc -l

ls: -l: No such file or directory
    9245


In [187]:
!ls "./data/ruska/faceswap/images/faces" -l |wc -l

ls: -l: No such file or directory
    7520


In [188]:
!ls "./data/ruska/neuraltextures/images/faces" -l |wc -l

ls: -l: No such file or directory
    7512


In [199]:
#print(facenames_3fakes[0].split('/')[7])

path_dict = {'Deepfakes': "./data/ruska/deepfakes/faces/", 
             'FaceSwap': "./data/ruska/faceswap/images/faces/",
             'NeuralTextures': "./data/ruska/neuraltextures/images/faces/"}

def get_feature_array_from_image_list_3fakepath(imgpathlist):
    imgpathlist_colab = []
    images_resized = []
    features_all = []
    k = 0
    for img in tqdm(imgpathlist):
        try:
            newpath = path_dict[img.split('/')[7]]
        except KeyError:
            k +=1
            print(img)
            print(k)
            continue
        img_new = get_new_path_for_image(img, newpath)
        img_cv2 = cv2.imread(img_new)[...,::-1]
        img_cv2_res = cv2.resize(img_cv2, (128, 128), interpolation = cv2.INTER_AREA)
        feature_vector = get_feature_vector(img_cv2_res)
        if len(feature_vector) == 0:
          continue
        features_all.append(feature_vector)
        imgpathlist_colab.append(img)
    return imgpathlist_colab, np.asarray(features_all)

In [163]:
!pwd

/Users/ruska/Documents/UCU/ML/course_project


In [169]:
!ls "./real_images/faces" -l |wc -l

ls: -l: No such file or directory
   17708


In [166]:
dataset = pd.read_csv('./dataset/dataset_tvt.csv', index_col = 0)

In [168]:
dataset_dropnan = dataset[~dataset['face_path_new'].isnull()]
dataset_real = dataset_dropnan[dataset_dropnan['fake_real_name'].isin(['real_images', 'real_images_youtube'])]
real_facenames = dataset_real['face_path_new'].tolist()
print(len(real_facenames))

11487


In [200]:
fake3_img_list, fake3_features_arr = get_feature_array_from_image_list_3fakepath(facenames_3fakes)
fake3_df = pd.DataFrame(fake3_features_arr)
fake3_df['path_drive'] = fake3_img_list
fake3_df.to_csv("./dataset/fake3_train_test_val.csv")




  0%|          | 0/21152 [00:00<?, ?it/s]


  0%|          | 1/21152 [00:00<48:11,  7.31it/s]


  0%|          | 3/21152 [00:00<43:52,  8.03it/s]


  0%|          | 5/21152 [00:00<36:05,  9.77it/s]


  0%|          | 8/21152 [00:00<29:34, 11.91it/s]


  0%|          | 10/21152 [00:00<27:38, 12.75it/s]


  0%|          | 13/21152 [00:00<23:42, 14.86it/s]


  0%|          | 16/21152 [00:00<20:51, 16.89it/s]


  0%|          | 19/21152 [00:01<19:08, 18.40it/s]


  0%|          | 22/21152 [00:01<17:45, 19.82it/s]


  0%|          | 25/21152 [00:01<19:04, 18.45it/s]


  0%|          | 27/21152 [00:01<18:44, 18.79it/s]


  0%|          | 30/21152 [00:01<17:46, 19.81it/s]


  0%|          | 33/21152 [00:01<16:58, 20.73it/s]


  0%|          | 36/21152 [00:01<16:38, 21.14it/s]


  0%|          | 39/21152 [00:01<15:51, 22.20it/s]


  0%|          | 43/21152 [00:02<14:37, 24.07it/s]


  0%|          | 46/21152 [00:02<14:17, 24.61it/s]


  0%|          | 49/21152 [00:02<16:18, 21.57it/s]


  0

  3%|▎         | 702/21152 [00:35<13:13, 25.78it/s]


  3%|▎         | 705/21152 [00:35<12:55, 26.37it/s]


  3%|▎         | 708/21152 [00:35<13:12, 25.78it/s]


  3%|▎         | 711/21152 [00:35<14:15, 23.89it/s]


  3%|▎         | 714/21152 [00:35<13:38, 24.98it/s]


  3%|▎         | 717/21152 [00:35<13:41, 24.88it/s]


  3%|▎         | 720/21152 [00:35<13:23, 25.42it/s]


  3%|▎         | 723/21152 [00:35<12:59, 26.21it/s]


  3%|▎         | 726/21152 [00:35<12:48, 26.59it/s]


  3%|▎         | 729/21152 [00:36<12:44, 26.73it/s]


  3%|▎         | 732/21152 [00:36<12:31, 27.16it/s]


  3%|▎         | 735/21152 [00:36<14:54, 22.82it/s]


  3%|▎         | 738/21152 [00:36<14:05, 24.13it/s]


  4%|▎         | 741/21152 [00:36<13:54, 24.47it/s]


  4%|▎         | 744/21152 [00:36<13:30, 25.19it/s]


  4%|▎         | 747/21152 [00:36<13:25, 25.32it/s]


  4%|▎         | 750/21152 [00:36<13:17, 25.59it/s]


  4%|▎         | 753/21152 [00:37<13:20, 25.49it/s]


  4%|▎         | 756/21152 [

  6%|▋         | 1325/21152 [01:10<17:32, 18.85it/s]


  6%|▋         | 1328/21152 [01:10<18:32, 17.82it/s]


  6%|▋         | 1330/21152 [01:10<19:19, 17.10it/s]


  6%|▋         | 1333/21152 [01:10<18:01, 18.32it/s]


  6%|▋         | 1335/21152 [01:10<17:46, 18.58it/s]


  6%|▋         | 1338/21152 [01:11<16:53, 19.56it/s]


  6%|▋         | 1341/21152 [01:11<15:46, 20.93it/s]


  6%|▋         | 1344/21152 [01:11<18:37, 17.73it/s]


  6%|▋         | 1347/21152 [01:11<17:05, 19.31it/s]


  6%|▋         | 1350/21152 [01:11<16:54, 19.51it/s]


  6%|▋         | 1353/21152 [01:11<15:31, 21.25it/s]


  6%|▋         | 1356/21152 [01:11<14:42, 22.44it/s]


  6%|▋         | 1359/21152 [01:11<14:01, 23.51it/s]


  6%|▋         | 1362/21152 [01:12<13:27, 24.50it/s]


  6%|▋         | 1365/21152 [01:12<13:39, 24.13it/s]


  6%|▋         | 1368/21152 [01:12<21:56, 15.02it/s]


  6%|▋         | 1371/21152 [01:12<19:25, 16.98it/s]


  6%|▋         | 1374/21152 [01:12<17:12, 19.16it/s]


  7%|▋    

/content/drive/My Drive/FakeImageDetector/data/manipulated_sequences/Face2Face/c23/faces/062_066.mp4_6.jpg_fd_1.jpg
1





  8%|▊         | 1732/21152 [01:28<12:19, 26.27it/s]


  8%|▊         | 1735/21152 [01:28<12:10, 26.59it/s]


  8%|▊         | 1738/21152 [01:28<12:59, 24.90it/s]


  8%|▊         | 1741/21152 [01:28<14:30, 22.29it/s]


  8%|▊         | 1744/21152 [01:28<13:45, 23.52it/s]


  8%|▊         | 1747/21152 [01:28<13:41, 23.63it/s]


  8%|▊         | 1750/21152 [01:28<13:02, 24.78it/s]


  8%|▊         | 1753/21152 [01:28<12:38, 25.56it/s]


  8%|▊         | 1756/21152 [01:29<12:31, 25.82it/s]


  8%|▊         | 1759/21152 [01:29<12:08, 26.62it/s]


  8%|▊         | 1762/21152 [01:29<12:02, 26.84it/s]


  8%|▊         | 1765/21152 [01:29<14:19, 22.57it/s]


  8%|▊         | 1768/21152 [01:29<14:02, 23.00it/s]


  8%|▊         | 1771/21152 [01:29<13:23, 24.13it/s]


  8%|▊         | 1774/21152 [01:29<12:57, 24.94it/s]


  8%|▊         | 1777/21152 [01:29<12:43, 25.39it/s]


  8%|▊         | 1780/21152 [01:29<12:30, 25.80it/s]


  8%|▊         | 1783/21152 [01:30<12:16, 26.32it/s]


  8%|▊ 

...

 86%|████████▌ | 18181/21152 [13:04<02:01, 24.45it/s]


 86%|████████▌ | 18184/21152 [13:04<02:16, 21.81it/s]


 86%|████████▌ | 18187/21152 [13:04<03:08, 15.72it/s]


 86%|████████▌ | 18189/21152 [13:04<03:05, 16.00it/s]


 86%|████████▌ | 18192/21152 [13:04<02:45, 17.87it/s]


 86%|████████▌ | 18195/21152 [13:04<02:29, 19.72it/s]


 86%|████████▌ | 18198/21152 [13:05<02:19, 21.16it/s]


 86%|████████▌ | 18201/21152 [13:05<02:13, 22.07it/s]


 86%|████████▌ | 18204/21152 [13:05<02:31, 19.46it/s]


 86%|████████▌ | 18207/21152 [13:05<02:17, 21.36it/s]


 86%|████████▌ | 18210/21152 [13:05<02:09, 22.74it/s]


 86%|████████▌ | 18213/21152 [13:05<02:05, 23.40it/s]


 86%|████████▌ | 18216/21152 [13:05<02:02, 23.99it/s]


 86%|████████▌ | 18219/21152 [13:05<02:00, 24.32it/s]


 86%|████████▌ | 18222/21152 [13:06<01:59, 24.53it/s]


 86%|████████▌ | 18225/21152 [13:06<02:00, 24.19it/s]


 86%|████████▌ | 18228/21152 [13:06<02:17, 21.29it/s]


 86%|████████▌ | 18231/21152 [13:06<02:08, 22.66

 89%|████████▉ | 18926/21152 [13:37<01:36, 22.98it/s]


 89%|████████▉ | 18929/21152 [13:37<01:33, 23.80it/s]


 90%|████████▉ | 18932/21152 [13:37<01:29, 24.91it/s]


 90%|████████▉ | 18935/21152 [13:37<01:26, 25.66it/s]


 90%|████████▉ | 18938/21152 [13:38<01:24, 26.33it/s]


 90%|████████▉ | 18941/21152 [13:38<01:22, 26.94it/s]


 90%|████████▉ | 18944/21152 [13:38<01:33, 23.51it/s]


 90%|████████▉ | 18947/21152 [13:38<01:31, 24.02it/s]


 90%|████████▉ | 18950/21152 [13:38<01:31, 24.13it/s]


 90%|████████▉ | 18953/21152 [13:38<01:29, 24.46it/s]


 90%|████████▉ | 18956/21152 [13:38<01:26, 25.33it/s]


 90%|████████▉ | 18959/21152 [13:38<01:25, 25.68it/s]


 90%|████████▉ | 18962/21152 [13:39<01:23, 26.36it/s]


 90%|████████▉ | 18965/21152 [13:39<01:22, 26.53it/s]


 90%|████████▉ | 18968/21152 [13:39<01:21, 26.82it/s]


 90%|████████▉ | 18971/21152 [13:39<01:32, 23.64it/s]


 90%|████████▉ | 18974/21152 [13:39<01:27, 24.88it/s]


 90%|████████▉ | 18977/21152 [13:39<01:28, 24.63

 93%|█████████▎| 19687/21152 [14:10<01:02, 23.48it/s]


 93%|█████████▎| 19690/21152 [14:10<01:13, 20.01it/s]


 93%|█████████▎| 19693/21152 [14:10<01:07, 21.59it/s]


 93%|█████████▎| 19696/21152 [14:10<01:03, 22.94it/s]


 93%|█████████▎| 19699/21152 [14:10<01:00, 24.21it/s]


 93%|█████████▎| 19702/21152 [14:10<00:58, 24.91it/s]


 93%|█████████▎| 19705/21152 [14:10<00:57, 25.36it/s]


 93%|█████████▎| 19708/21152 [14:11<00:55, 26.19it/s]


 93%|█████████▎| 19711/21152 [14:11<00:53, 26.78it/s]


 93%|█████████▎| 19714/21152 [14:11<01:00, 23.73it/s]


 93%|█████████▎| 19717/21152 [14:11<00:58, 24.72it/s]


 93%|█████████▎| 19720/21152 [14:11<00:55, 26.02it/s]


 93%|█████████▎| 19723/21152 [14:11<00:54, 26.33it/s]


 93%|█████████▎| 19726/21152 [14:11<00:53, 26.63it/s]


 93%|█████████▎| 19729/21152 [14:11<00:54, 26.20it/s]


 93%|█████████▎| 19732/21152 [14:12<00:53, 26.68it/s]


 93%|█████████▎| 19735/21152 [14:12<00:53, 26.57it/s]


 93%|█████████▎| 19738/21152 [14:12<00:52, 26.96

 97%|█████████▋| 20441/21152 [14:42<00:35, 20.19it/s]


 97%|█████████▋| 20444/21152 [14:43<00:33, 21.25it/s]


 97%|█████████▋| 20447/21152 [14:43<00:31, 22.48it/s]


 97%|█████████▋| 20450/21152 [14:43<00:36, 19.46it/s]


 97%|█████████▋| 20453/21152 [14:43<00:33, 21.02it/s]


 97%|█████████▋| 20456/21152 [14:43<00:33, 20.60it/s]


 97%|█████████▋| 20459/21152 [14:43<00:32, 21.57it/s]


 97%|█████████▋| 20462/21152 [14:43<00:29, 23.04it/s]


 97%|█████████▋| 20465/21152 [14:43<00:28, 23.93it/s]


 97%|█████████▋| 20468/21152 [14:44<00:27, 24.62it/s]


 97%|█████████▋| 20471/21152 [14:44<00:27, 25.17it/s]


 97%|█████████▋| 20474/21152 [14:44<00:31, 21.49it/s]


 97%|█████████▋| 20478/21152 [14:44<00:28, 23.86it/s]


 97%|█████████▋| 20481/21152 [14:44<00:26, 25.33it/s]


 97%|█████████▋| 20484/21152 [14:44<00:26, 25.12it/s]


 97%|█████████▋| 20487/21152 [14:44<00:26, 25.14it/s]


 97%|█████████▋| 20490/21152 [14:44<00:26, 25.18it/s]


 97%|█████████▋| 20493/21152 [14:45<00:25, 25.47

/content/drive/My Drive/FakeImageDetector/data/original_sequences/faces/349.mp4_image4.jpg_fd_1.jpg
19


In [175]:
real_img_list, real_features_arr = get_feature_array_from_image_list_newpath(real_facenames, "./real_images/faces/")
real_df = pd.DataFrame(real_features_arr)
real_df['path_drive'] = real_img_list
real_df.to_csv("./dataset/real_train_test_val.csv")



  0%|          | 0/11487 [00:00<?, ?it/s]

  0%|          | 1/11487 [00:01<4:48:53,  1.51s/it]

  0%|          | 2/11487 [00:01<3:31:06,  1.10s/it]

  0%|          | 4/11487 [00:01<2:34:01,  1.24it/s]

  0%|          | 6/11487 [00:02<1:51:10,  1.72it/s]

  0%|          | 9/11487 [00:02<1:20:33,  2.37it/s]

  0%|          | 12/11487 [00:02<58:47,  3.25it/s] 

  0%|          | 14/11487 [00:02<44:00,  4.34it/s]

  0%|          | 16/11487 [00:02<34:04,  5.61it/s]

  0%|          | 18/11487 [00:02<26:46,  7.14it/s]

  0%|          | 21/11487 [00:02<21:03,  9.07it/s]

  0%|          | 23/11487 [00:02<18:52, 10.12it/s]

  0%|          | 25/11487 [00:02<16:40, 11.46it/s]

  0%|          | 28/11487 [00:03<14:11, 13.46it/s]

  0%|          | 30/11487 [00:03<12:48, 14.91it/s]

  0%|          | 33/11487 [00:03<11:37, 16.43it/s]

  0%|          | 36/11487 [00:03<10:17, 18.55it/s]

  0%|          | 39/11487 [00:03<09:48, 19.47it/s]

  0%|          | 42/11487 [00:03<09:11, 20.76it/s]

  0%|        

  7%|▋         | 850/11487 [00:36<07:10, 24.68it/s]

  7%|▋         | 853/11487 [00:36<07:06, 24.96it/s]

  7%|▋         | 856/11487 [00:36<06:54, 25.62it/s]

  7%|▋         | 859/11487 [00:36<06:44, 26.25it/s]

  8%|▊         | 862/11487 [00:36<06:39, 26.58it/s]

  8%|▊         | 865/11487 [00:36<06:37, 26.75it/s]

  8%|▊         | 868/11487 [00:36<07:48, 22.67it/s]

  8%|▊         | 871/11487 [00:36<07:33, 23.39it/s]

  8%|▊         | 874/11487 [00:37<07:15, 24.35it/s]

  8%|▊         | 877/11487 [00:37<06:59, 25.32it/s]

  8%|▊         | 880/11487 [00:37<06:49, 25.90it/s]

  8%|▊         | 883/11487 [00:37<06:43, 26.27it/s]

  8%|▊         | 886/11487 [00:37<06:37, 26.67it/s]

  8%|▊         | 889/11487 [00:37<06:32, 27.00it/s]

  8%|▊         | 892/11487 [00:37<06:29, 27.20it/s]

  8%|▊         | 895/11487 [00:37<07:42, 22.91it/s]

  8%|▊         | 898/11487 [00:38<07:24, 23.81it/s]

  8%|▊         | 901/11487 [00:38<07:14, 24.35it/s]

  8%|▊         | 904/11487 [00:38<06:59, 25.24

 15%|█▍        | 1684/11487 [01:11<05:53, 27.76it/s]

 15%|█▍        | 1688/11487 [01:11<05:57, 27.39it/s]

 15%|█▍        | 1692/11487 [01:11<05:44, 28.39it/s]

 15%|█▍        | 1696/11487 [01:12<07:03, 23.12it/s]

 15%|█▍        | 1699/11487 [01:12<07:05, 22.98it/s]

 15%|█▍        | 1702/11487 [01:12<07:02, 23.15it/s]

 15%|█▍        | 1705/11487 [01:12<07:01, 23.22it/s]

 15%|█▍        | 1708/11487 [01:12<06:46, 24.03it/s]

 15%|█▍        | 1711/11487 [01:12<06:35, 24.69it/s]

 15%|█▍        | 1714/11487 [01:12<06:24, 25.40it/s]

 15%|█▍        | 1717/11487 [01:12<07:30, 21.67it/s]

 15%|█▍        | 1720/11487 [01:13<06:59, 23.26it/s]

 15%|█▍        | 1723/11487 [01:13<06:49, 23.83it/s]

 15%|█▌        | 1726/11487 [01:13<06:34, 24.71it/s]

 15%|█▌        | 1729/11487 [01:13<06:26, 25.25it/s]

 15%|█▌        | 1732/11487 [01:13<06:24, 25.36it/s]

 15%|█▌        | 1735/11487 [01:13<06:21, 25.58it/s]

 15%|█▌        | 1738/11487 [01:13<06:10, 26.28it/s]

 15%|█▌        | 1741/11487 

 22%|██▏       | 2530/11487 [01:45<05:45, 25.91it/s]

 22%|██▏       | 2533/11487 [01:45<05:42, 26.15it/s]

 22%|██▏       | 2536/11487 [01:45<05:35, 26.64it/s]

 22%|██▏       | 2539/11487 [01:45<06:25, 23.24it/s]

 22%|██▏       | 2542/11487 [01:46<06:19, 23.57it/s]

 22%|██▏       | 2545/11487 [01:46<06:06, 24.43it/s]

 22%|██▏       | 2548/11487 [01:46<06:16, 23.73it/s]

 22%|██▏       | 2551/11487 [01:46<06:02, 24.62it/s]

 22%|██▏       | 2554/11487 [01:46<05:51, 25.42it/s]

 22%|██▏       | 2558/11487 [01:46<05:17, 28.16it/s]

 22%|██▏       | 2561/11487 [01:46<05:18, 28.05it/s]

 22%|██▏       | 2564/11487 [01:46<05:55, 25.09it/s]

 22%|██▏       | 2567/11487 [01:46<06:02, 24.58it/s]

 22%|██▏       | 2570/11487 [01:47<05:50, 25.41it/s]

 22%|██▏       | 2573/11487 [01:47<05:38, 26.30it/s]

 22%|██▏       | 2576/11487 [01:47<05:36, 26.44it/s]

 22%|██▏       | 2579/11487 [01:47<05:36, 26.51it/s]

 22%|██▏       | 2582/11487 [01:47<05:34, 26.63it/s]

 23%|██▎       | 2585/11487 

 29%|██▉       | 3386/11487 [02:18<05:12, 25.95it/s]

 30%|██▉       | 3389/11487 [02:18<05:09, 26.16it/s]

 30%|██▉       | 3392/11487 [02:18<05:51, 23.05it/s]

 30%|██▉       | 3395/11487 [02:19<05:37, 23.95it/s]

 30%|██▉       | 3398/11487 [02:19<05:34, 24.17it/s]

 30%|██▉       | 3401/11487 [02:19<05:22, 25.09it/s]

 30%|██▉       | 3404/11487 [02:19<05:11, 25.91it/s]

 30%|██▉       | 3407/11487 [02:19<05:09, 26.07it/s]

 30%|██▉       | 3410/11487 [02:19<05:02, 26.73it/s]

 30%|██▉       | 3413/11487 [02:19<04:58, 27.05it/s]

 30%|██▉       | 3416/11487 [02:19<05:28, 24.59it/s]

 30%|██▉       | 3419/11487 [02:19<05:26, 24.72it/s]

 30%|██▉       | 3422/11487 [02:20<05:16, 25.52it/s]

 30%|██▉       | 3425/11487 [02:20<05:14, 25.63it/s]

 30%|██▉       | 3428/11487 [02:20<05:05, 26.41it/s]

 30%|██▉       | 3431/11487 [02:20<05:05, 26.41it/s]

 30%|██▉       | 3434/11487 [02:20<05:07, 26.16it/s]

 30%|██▉       | 3437/11487 [02:20<05:03, 26.50it/s]

 30%|██▉       | 3440/11487 

...

 73%|███████▎  | 8349/11487 [05:35<02:08, 24.36it/s]

 73%|███████▎  | 8352/11487 [05:35<02:06, 24.82it/s]

 73%|███████▎  | 8355/11487 [05:35<02:02, 25.67it/s]

 73%|███████▎  | 8358/11487 [05:35<01:57, 26.59it/s]

 73%|███████▎  | 8361/11487 [05:35<01:55, 27.13it/s]

 73%|███████▎  | 8364/11487 [05:35<01:53, 27.40it/s]

 73%|███████▎  | 8367/11487 [05:35<01:53, 27.44it/s]

 73%|███████▎  | 8370/11487 [05:35<02:07, 24.39it/s]

 73%|███████▎  | 8373/11487 [05:35<02:07, 24.46it/s]

 73%|███████▎  | 8376/11487 [05:36<02:02, 25.46it/s]

 73%|███████▎  | 8379/11487 [05:36<02:01, 25.67it/s]

 73%|███████▎  | 8382/11487 [05:36<01:57, 26.35it/s]

 73%|███████▎  | 8385/11487 [05:36<01:59, 25.92it/s]

 73%|███████▎  | 8388/11487 [05:36<01:55, 26.73it/s]

 73%|███████▎  | 8391/11487 [05:36<01:53, 27.26it/s]

 73%|███████▎  | 8394/11487 [05:36<01:52, 27.48it/s]

 73%|███████▎  | 8397/11487 [05:36<02:07, 24.22it/s]

 73%|███████▎  | 8400/11487 [05:37<02:06, 24.45it/s]

 73%|███████▎  | 8403/11487 

 80%|███████▉  | 9167/11487 [06:08<01:37, 23.81it/s]

 80%|███████▉  | 9170/11487 [06:08<01:34, 24.44it/s]

 80%|███████▉  | 9173/11487 [06:08<01:31, 25.28it/s]

 80%|███████▉  | 9177/11487 [06:08<01:24, 27.40it/s]

 80%|███████▉  | 9181/11487 [06:08<01:19, 29.11it/s]

 80%|███████▉  | 9184/11487 [06:08<01:19, 28.82it/s]

 80%|███████▉  | 9187/11487 [06:08<01:36, 23.73it/s]

 80%|████████  | 9190/11487 [06:09<01:35, 23.98it/s]

 80%|████████  | 9193/11487 [06:09<01:33, 24.52it/s]

 80%|████████  | 9196/11487 [06:09<01:31, 25.04it/s]

 80%|████████  | 9199/11487 [06:09<01:30, 25.40it/s]

 80%|████████  | 9202/11487 [06:09<01:27, 26.03it/s]

 80%|████████  | 9205/11487 [06:09<01:25, 26.77it/s]

 80%|████████  | 9208/11487 [06:09<01:24, 26.90it/s]

 80%|████████  | 9211/11487 [06:09<01:25, 26.73it/s]

 80%|████████  | 9214/11487 [06:09<01:39, 22.90it/s]

 80%|████████  | 9217/11487 [06:10<01:38, 23.08it/s]

 80%|████████  | 9220/11487 [06:10<01:36, 23.57it/s]

 80%|████████  | 9223/11487 

 87%|████████▋ | 9999/11487 [06:41<01:00, 24.52it/s]

 87%|████████▋ | 10002/11487 [06:41<00:58, 25.29it/s]

 87%|████████▋ | 10005/11487 [06:41<00:56, 26.03it/s]

 87%|████████▋ | 10008/11487 [06:41<00:55, 26.49it/s]

 87%|████████▋ | 10011/11487 [06:41<00:54, 26.91it/s]

 87%|████████▋ | 10014/11487 [06:41<01:04, 22.85it/s]

 87%|████████▋ | 10017/11487 [06:42<01:01, 23.79it/s]

 87%|████████▋ | 10020/11487 [06:42<01:00, 24.39it/s]

 87%|████████▋ | 10023/11487 [06:42<00:58, 24.90it/s]

 87%|████████▋ | 10026/11487 [06:42<00:57, 25.26it/s]

 87%|████████▋ | 10029/11487 [06:42<00:56, 25.66it/s]

 87%|████████▋ | 10033/11487 [06:42<00:53, 27.22it/s]

 87%|████████▋ | 10036/11487 [06:42<00:54, 26.78it/s]

 87%|████████▋ | 10039/11487 [06:42<01:04, 22.43it/s]

 87%|████████▋ | 10042/11487 [06:43<01:02, 23.01it/s]

 87%|████████▋ | 10045/11487 [06:43<01:01, 23.53it/s]

 87%|████████▋ | 10048/11487 [06:43<00:59, 24.27it/s]

 87%|████████▋ | 10051/11487 [06:43<00:56, 25.25it/s]

 88%|██████

 94%|█████████▎| 10765/11487 [07:16<00:34, 20.79it/s]

 94%|█████████▎| 10768/11487 [07:16<00:33, 21.59it/s]

 94%|█████████▍| 10771/11487 [07:16<00:30, 23.15it/s]

 94%|█████████▍| 10774/11487 [07:16<00:29, 24.20it/s]

 94%|█████████▍| 10777/11487 [07:16<00:28, 24.85it/s]

 94%|█████████▍| 10780/11487 [07:16<00:27, 25.40it/s]

 94%|█████████▍| 10783/11487 [07:16<00:27, 25.66it/s]

 94%|█████████▍| 10786/11487 [07:16<00:30, 22.84it/s]

 94%|█████████▍| 10789/11487 [07:17<00:29, 23.93it/s]

 94%|█████████▍| 10792/11487 [07:17<00:28, 24.25it/s]

 94%|█████████▍| 10795/11487 [07:17<00:27, 24.98it/s]

 94%|█████████▍| 10798/11487 [07:17<00:27, 25.36it/s]

 94%|█████████▍| 10801/11487 [07:17<00:26, 25.87it/s]

 94%|█████████▍| 10805/11487 [07:17<00:24, 27.45it/s]

 94%|█████████▍| 10808/11487 [07:17<00:33, 20.52it/s]

 94%|█████████▍| 10812/11487 [07:17<00:29, 23.18it/s]

 94%|█████████▍| 10815/11487 [07:18<00:27, 24.24it/s]

 94%|█████████▍| 10818/11487 [07:18<00:27, 24.68it/s]

 94%|█████

100%|█████████▉| 11484/11487 [07:50<00:00, 24.70it/s]

100%|██████████| 11487/11487 [07:50<00:00, 24.42it/s]


In [176]:
print(real_df.shape)

(11187, 19201)


In [180]:
dataset_dropnan = dataset[~dataset['face_path_new'].isnull()]
dataset_3fakes = dataset_dropnan[dataset_dropnan['fake_real_name'].isin(['Deepfakes', 'NeuralTextures', 'FaceSwap'])]
facenames_3fakes = dataset_3fakes['face_path_new'].tolist()
print(len(facenames_3fakes))

21152


In [185]:
print(facenames_3fakes[0].split('/')[7])

NeuralTextures


In [3]:
dataset = pd.read_csv('./dataset/dataset_tvt.csv', index_col = 0)

In [202]:
print(fake3_df.head())

     0    1    2    3    4    5    6    7    8    9  ...  19191  19192  19193  \
0  135   93   69  138   96   72  141   99   75  147  ...    138     92     59   
1  163  122  127  166  123  128  169  124  128  173  ...    172    121    117   
2   12   12   12   13   15   14   14   16   15   15  ...    195    127    108   
3   53   43   38   63   52   46   73   60   54   83  ...    142    105     89   
4  144   97   87  145   99   87  146   99   85  147  ...    133     87     81   

   19194  19195  19196  19197  19198  19199  \
0    137     91     57    137     91     57   
1    173    123    119    172    123    119   
2    194    126    107    193    125    106   
3    141    104     88    141    104     88   
4    134     88     82    132     87     83   

                                          path_drive  
0  /content/drive/My Drive/FakeImageDetector/data...  
1  /content/drive/My Drive/FakeImageDetector/data...  
2  /content/drive/My Drive/FakeImageDetector/data...  
3  /conten

In [4]:
real_df = pd.read_csv('./dataset/real_train_test_val.csv', index_col = 0)
print(real_df.head())

     0    1    2    3    4    5    6    7    8    9  ...  19191  19192  19193  \
0  125  124  120  126  125  121  128  124  121  127  ...    134    104     76   
1  230  204  181  228  202  180  228  203  181  230  ...     92     61     43   
2  221  191  166  223  193  167  221  192  169  220  ...     90     69     46   
3  223  192  163  223  193  164  223  193  165  221  ...     98     66     53   
4  224  191  155  225  193  158  226  195  162  227  ...     89     45     38   

   19194  19195  19196  19197  19198  19199  \
0    127    100     72    113     89     61   
1     91     60     41     92     63     45   
2     93     73     49     93     71     48   
3     98     66     53     98     66     53   
4     94     51     44     98     59     51   

                                          path_drive  
0  /content/drive/My Drive/FakeImageDetector/data...  
1  /content/drive/My Drive/FakeImageDetector/data...  
2  /content/drive/My Drive/FakeImageDetector/data...  
3  /conten

In [5]:
face2face_df = pd.read_csv('./dataset/face2face_train_test_val.csv', index_col = 0)

In [6]:
fake3_df = pd.read_csv('./dataset/fake3_train_test_val.csv', index_col = 0)

In [206]:
print(face2face_df.head())

     0    1    2    3    4    5    6    7    8    9  ...  19192  19193  19194  \
0  222  144  122  222  144  122  222  144  123  222  ...    112    105    183   
1  120   83   74  104   67   58   94   56   47   91  ...    110     91    163   
2  199  144  127  199  144  127  200  145  128  200  ...    117     97    170   
3  127   73   39  132   77   42  135   78   41  133  ...    101     69    161   
4   65   50   42   94   69   59  127   94   83  128  ...    133    109    186   

   19195  19196  19197  19198  19199  \
0    112    104    180    111    102   
1    106     89    160    106     91   
2    115     95    167    112     92   
3     98     67    153     91     61   
4    132    108    184    130    106   

                                                path  train  
0  /content/drive/My Drive/FakeImageDetector/data...  train  
1  /content/drive/My Drive/FakeImageDetector/data...  train  
2  /content/drive/My Drive/FakeImageDetector/data...  train  
3  /content/drive/My Dri

In [7]:
face2face_df = face2face_df.rename({'path': 'path_drive'}, axis=1)

In [9]:
dataset_dropnan = dataset[~dataset['face_path_new'].isnull()]

In [10]:
dataset_path_train = dataset_dropnan.loc[:, ['face_path_new', 'train']]

In [225]:
sub = dataset_path_train.iloc[:10,:]
print(sub.shape)

(10, 2)


In [11]:
train_path_dict = dataset_path_train.set_index('face_path_new')['train'].to_dict()


In [232]:
train_path_dict = sub.set_index('face_path_new')['train'].to_dict()
print(train_path_dict)
# for k,v in train_path_dict.items():
#     print(v)

{'/content/drive/My Drive/FakeImageDetector/data/original_sequences/faces/21__outside_talking_still_laughing.mp4_image6.jpg_fd_1.jpg': 'train', '/content/drive/My Drive/FakeImageDetector/data/original_sequences/faces/21__outside_talking_still_laughing.mp4_image7.jpg_fd_1.jpg': 'train', '/content/drive/My Drive/FakeImageDetector/data/original_sequences/faces/21__outside_talking_still_laughing.mp4_image8.jpg_fd_1.jpg': 'train', '/content/drive/My Drive/FakeImageDetector/data/original_sequences/faces/21__outside_talking_still_laughing.mp4_image9.jpg_fd_1.jpg': 'train', '/content/drive/My Drive/FakeImageDetector/data/original_sequences/faces/21__outside_talking_still_laughing.mp4_image10.jpg_fd_1.jpg': 'train', '/content/drive/My Drive/FakeImageDetector/data/original_sequences/faces/21__outside_talking_still_laughing.mp4_image12.jpg_fd_1.jpg': 'train', '/content/drive/My Drive/FakeImageDetector/data/original_sequences/faces/21__outside_talking_still_laughing.mp4_image13.jpg_fd_1.jpg': 'tra

In [222]:
d = face2face_df['path_drive'].loc[0].tolist()
print(d[0])

/content/drive/My Drive/FakeImageDetector/data/manipulated_sequences/Face2Face/c23/faces/169_227.mp4_5.jpg_fd_1.jpg


In [235]:
for i in d:
    print(train_path_dict[i])

train
test
validation


In [12]:
real_df['train']=real_df['path_drive'].apply(lambda x: train_path_dict[x])
print(real_df)

         0    1    2    3    4    5    6    7    8    9  ...  19192  19193  \
0      125  124  120  126  125  121  128  124  121  127  ...    104     76   
1      230  204  181  228  202  180  228  203  181  230  ...     61     43   
2      221  191  166  223  193  167  221  192  169  220  ...     69     46   
3      223  192  163  223  193  164  223  193  165  221  ...     66     53   
4      224  191  155  225  193  158  226  195  162  227  ...     45     38   
5      222  195  169  222  195  169  221  193  168  218  ...     63     44   
6      226  200  173  227  201  174  229  203  177  229  ...     65     51   
7      222  188  156  224  190  157  225  191  159  224  ...     51     32   
8      219  186  152  219  186  153  218  186  155  217  ...     60     49   
9      227  194  159  227  196  161  227  196  168  226  ...     67     43   
10     217  186  154  216  185  155  215  186  156  213  ...     58     37   
11     222  187  159  222  189  164  223  191  166  223  ...    

In [238]:
real_df_test=real_df[real_df['train']=='test']

In [13]:
real_df.groupby(['train']).agg('count')

,0,1,2,3,4,5,6,7,8,9,...,19191,19192,19193,19194,19195,19196,19197,19198,19199,path_drive
train,,,,,,,,,,,,,,,,,,,,,
test,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468,...,1468,1468,1468,1468,1468,1468,1468,1468,1468,1468
train,8503,8503,8503,8503,8503,8503,8503,8503,8503,8503,...,8503,8503,8503,8503,8503,8503,8503,8503,8503,8503
validation,1216,1216,1216,1216,1216,1216,1216,1216,1216,1216,...,1216,1216,1216,1216,1216,1216,1216,1216,1216,1216


In [14]:
fake3_df['train']=fake3_df['path_drive'].apply(lambda x: train_path_dict[x])

In [15]:
fake3_df.groupby(['train'])['path_drive'].agg('count')

train
test           4116
train         13456
validation     3379
Name: path_drive, dtype: int64

In [16]:
real_df.to_csv("./dataset/real_train_test_val.csv")

In [210]:
print(face2face_df.where(face2face_df['path_drive']==dataset['face_path_new']).notna())

ValueError: Can only compare identically-labeled Series objects

In [244]:
all2 = pd.concat([face2face_df, fake3_df], ignore_index=True)
print(all2.shape[0])

/Users/ruska/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


30158


In [245]:
print(face2face_df.shape)
print(fake3_df.shape)

(9207, 19202)
(20951, 19202)


In [246]:
features_df = pd.concat([real_df, all2], ignore_index=True)

/Users/ruska/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [19]:
#"content/drive/My Drive/FakeImageDetector/data/manipulated_sequences/Face2Face/c23/faces/169_227.mp4_5.jpg_fd_1.jpg"
def get_fake_real_folder(imgpth):
    if imgpth.split('/')[6] == 'original_sequences':
        return 'real_images'
    else:
        return imgpth.split('/')[7]

In [20]:
fake3_df['fake_real'] = fake3_df['path_drive'].apply(lambda x: get_fake_real_folder(x))

In [21]:
deepfakes = fake3_df[fake3_df['fake_real'] == 'Deepfakes']
print(deepfakes.shape)

(7760, 19203)


In [22]:
deepfakes.to_csv("./dataset/deepfakes_train_test_val.csv")

In [23]:
neuraltextures = fake3_df[fake3_df['fake_real'] == 'NeuralTextures']

In [24]:
neuraltextures.to_csv("./dataset/neuraltextures_train_test_val.csv")

In [25]:
faceswap = fake3_df[fake3_df['fake_real'] == 'FaceSwap']

In [27]:
faceswap.to_csv("./dataset/faceswap_train_test_val.csv")

In [26]:
print(faceswap.shape)

(6550, 19203)


In [ ]:
print(real_df['path_drive'][0])

In [208]:
print(face2face_df.head())

     0    1    2    3    4    5    6    7    8    9  ...  19192  19193  19194  \
0  222  144  122  222  144  122  222  144  123  222  ...    112    105    183   
1  120   83   74  104   67   58   94   56   47   91  ...    110     91    163   
2  199  144  127  199  144  127  200  145  128  200  ...    117     97    170   
3  127   73   39  132   77   42  135   78   41  133  ...    101     69    161   
4   65   50   42   94   69   59  127   94   83  128  ...    133    109    186   

   19195  19196  19197  19198  19199  \
0    112    104    180    111    102   
1    106     89    160    106     91   
2    115     95    167    112     92   
3     98     67    153     91     61   
4    132    108    184    130    106   

                                          path_drive  train  
0  /content/drive/My Drive/FakeImageDetector/data...  train  
1  /content/drive/My Drive/FakeImageDetector/data...  train  
2  /content/drive/My Drive/FakeImageDetector/data...  train  
3  /content/drive/My Dri

In [136]:
print(imgs)
print(len(imgs))

['023_923.mp4_11.jpg_fd_.jpg', '023_923.mp4_8.jpg_fd_.jpg', '023_923.mp4_1.jpg_fd_.jpg', '023_923.mp4_4.jpg_fd_.jpg', '595_597.mp4_5.jpg_fd_.jpg', '023_923.mp4_7.jpg_fd_.jpg', '023_923.mp4_2.jpg_fd_.jpg', '595_597.mp4_3.jpg_fd_.jpg', '023_923.mp4_12.jpg_fd_.jpg', '595_597.mp4_4.jpg_fd_.jpg', '023_923.mp4_5.jpg_fd_.jpg', '595_597.mp4_1.jpg_fd_.jpg', '023_923.mp4_10.jpg_fd_.jpg', '023_923.mp4_9.jpg_fd_.jpg', '023_923.mp4_13.jpg_fd_.jpg', '595_597.mp4_2.jpg_fd_.jpg', '023_923.mp4_3.jpg_fd_.jpg', '023_923.mp4_6.jpg_fd_.jpg']
18


In [137]:
print(featrs)
print(featrs.shape)

[[146 107  74 ... 155 122 105]
 [108  71  59 ... 175 130 109]
 [123  86  59 ... 163 127 103]
 ...
 [ 71  29  10 ... 162  88  59]
 [114  73  51 ... 170 126 107]
 [114  82  59 ... 170 129 107]]
(18, 19200)


In [139]:
df = pd.DataFrame(featrs)
df['path'] = imgs
print(df)

      0    1   2    3    4   5    6    7   8    9  ...  19191  19192  19193  \
0   146  107  74  146  107  74  146  107  74  146  ...    158    125    106   
1   108   71  59  108   71  55  110   73  56  112  ...    177    131    112   
2   123   86  59  122   85  58  123   84  58  124  ...    168    129    105   
3   119   86  71  124   90  71  126   91  70  127  ...    171    130    105   
4    75   38  23   81   42  29   88   47  36   96  ...    181     95     74   
5   119   79  58  119   78  57  120   79  57  121  ...    177    137    112   
6   127   87  62  126   86  60  126   87  59  126  ...    170    129    111   
7    68   30   9   71   32  12   74   35  15   76  ...    148     81     48   
8   130   89  68  132   91  69  134   92  70  136  ...    167    129    108   
9    66   29  13   69   32  16   71   34  18   74  ...    172     95     68   
10  118   84  59  120   86  59  122   87  60  124  ...    173    131    109   
11   68   28  12   70   28  16   73   30  21   75  .

In [143]:
print(df.iloc[:,:-1].values.shape)

(18, 19200)


In [ ]:
nd_array = np.reshape(flat_array, (40, 40, 4, 3))
np.reshape(arr, (n,3))

In [144]:
def demeaning_local(array_nd):
    means = array_nd.mean(axis=(0,1), dtype='float64')
    print('Means: %s' % means)
    array_nd_0mean -= means
    return array_nd_0mean

In [ ]:
def demeaning_global(arr):
    mean = arr.mean()
    print('Mean: %.3f' % mean)
    arr_0mean = arr - mean
    return arr_0mean
    